In [ ]:
###########import packages##########
import tensorflow as tf
import keras
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
#from keras.backend.tensorflow_backend import set_session
from keras import optimizers
from keras import regularizers
from keras import backend as K
from keras.callbacks import EarlyStopping
from keras.callbacks import TensorBoard
#from keras.constraints import max_norm
from keras.models import Sequential 
from keras.layers import Dense 
from keras.layers import Dropout 
from keras.layers import BatchNormalization
from keras.wrappers.scikit_learn import KerasClassifier 
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import GridSearchCV
#%matplotlib

###########fix random seed for reproducability##########
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

'''
gpu_options = tf.GPUOptions(allow_growth=True)
gpu_options =tf.GPUOptions(per_process_gpu_memory_fraction=0.8,allow_growth=True) ##每个gpu占用0.8存
config=tf.ConfigProto(gpu_options=gpu_options,allow_soft_placement=True)
sess=tf.Session(config=config)
'''
seed=1
np.random.seed(seed)
###########wrapping root mean square error for later calls##########
def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())
###########loading data##########
csv = "../../database/database124.csv"

fdata=pd.read_csv(csv, index_col=0)
raw_data = fdata.iloc[:, 1:]

median_raw_data=raw_data.median()
dict_median_raw_data=median_raw_data.to_dict()
data=raw_data.fillna(dict_median_raw_data)
#print(data)
#print(np.std(data,axis=0))
###########data standardization##########
standardized_data = (data-np.mean(data,axis=0))/np.std(data,axis=0)
#standardized_data = (data-data.mean(axis=0))/data.std(axis=0)
raw_param=standardized_data.iloc[:,0:121]
raw_power=standardized_data.iloc[:,121]
#print(raw_param)
print(raw_power)
X=raw_param.values.astype(np.float32)
y=raw_power.values.astype(np.float32)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.15)

file = open('../../results/ANN_Opt_layer/alkaline-ANN-1-layer.txt','w')
dropout_list= np.arange(0, 1, 0.1)
for neurons in range(50,1050,50):
    ###########implementing hyperparameters##########
    neurons1=neurons
    ###########keras ANN model construction##########
    model = Sequential() 
    model.add(Dense(neurons1, input_dim=121, kernel_initializer='random_normal',
                bias_initializer='random_normal',activation='relu',kernel_regularizer=keras.regularizers.l2(0.0001))) 
    model.add(Dropout(0))
    model.add(Dense(neurons1, input_dim=neurons1, kernel_initializer='random_normal',
                bias_initializer='random_normal',activation='relu',kernel_regularizer=keras.regularizers.l2(0.0001))) 
    model.add(Dropout(0)) 
    model.add(Dense(1, input_dim=neurons1, activation='linear'))
    adam=tf.keras.optimizers.Adam(lr=0.002)
    model.compile(loss='mse', optimizer=adam) 
    #print('Training ------------')
    ###########train the model with the training set##########
    ###########testset has been remove before##########
    model.fit(X_train, y_train,verbose=0, epochs=200, batch_size=32,validation_split=0.1,callbacks=[TensorBoard(log_dir='mytensorboard')])
    loss= model.evaluate(X_test, y_test)
    predict_ann= model.predict(X_test)
    train_ann= model.predict(X_train)
    ###########result output##########
    x_prediction_maximum_power_ann=predict_ann*np.std(data,axis=0)[121]+np.mean(data,axis=0)[121]
    y_real_maximum_power=y_test*np.std(data,axis=0)[121]+np.mean(data,axis=0)[121]
    x_prediction_maximum_power_ann=x_prediction_maximum_power_ann[:,0]
    x_prediction_maximum_power_ann_series=pd.Series(x_prediction_maximum_power_ann)
    y_real_maximum_power_series=pd.Series(y_real_maximum_power)
    corr_ann = round(x_prediction_maximum_power_ann_series.corr(y_real_maximum_power_series), 4)
    rmse_val= rmse(x_prediction_maximum_power_ann,y_real_maximum_power)
    print('ANN,R2',corr_ann,'RMSE',rmse_val)
    layer_str = '%d'%1
    neurons_str = '%d'%neurons
    corr_ann_str = '%f'%corr_ann
    rmse_val_str = '%f'%rmse_val
    file.write(layer_str+' '+ neurons_str +' '+ corr_ann_str+' '+ rmse_val_str +'\n')
file.close() 

file = open('../../results/ANN_Opt_layer/alkaline-ANN-2-layer.txt','w')
dropout_list= np.arange(0, 1, 0.1)
for neurons in range(50,1050,50):
    ###########implementing hyperparameters##########
    neurons1=neurons
    ###########keras ANN model construction##########
    model = Sequential() 
    model.add(Dense(neurons1, input_dim=121, kernel_initializer='random_normal',
                bias_initializer='random_normal',activation='relu',kernel_regularizer=keras.regularizers.l2(0.0001))) 
    model.add(Dropout(0))
    model.add(Dense(neurons1, input_dim=neurons1, kernel_initializer='random_normal',
                bias_initializer='random_normal',activation='relu',kernel_regularizer=keras.regularizers.l2(0.0001))) 
    model.add(Dropout(0))
    model.add(Dense(neurons1, input_dim=neurons1, kernel_initializer='random_normal',
                bias_initializer='random_normal',activation='relu',kernel_regularizer=keras.regularizers.l2(0.0001))) 
    model.add(Dropout(0))
    model.add(Dense(1, input_dim=neurons1, activation='linear'))
    adam=tf.keras.optimizers.Adam(lr=0.002)
    model.compile(loss='mse', optimizer=adam) 
    #print('Training ------------')
    ###########train the model with the training set##########
    ###########testset has been remove before##########
    model.fit(X_train, y_train,verbose=0, epochs=200, batch_size=32,validation_split=0.1,callbacks=[TensorBoard(log_dir='mytensorboard')])
    loss= model.evaluate(X_test, y_test)
    predict_ann= model.predict(X_test)
    train_ann= model.predict(X_train)
    ###########result output##########
    x_prediction_maximum_power_ann=predict_ann*np.std(data,axis=0)[121]+np.mean(data,axis=0)[121]
    y_real_maximum_power=y_test*np.std(data,axis=0)[121]+np.mean(data,axis=0)[121]
    x_prediction_maximum_power_ann=x_prediction_maximum_power_ann[:,0]
    x_prediction_maximum_power_ann_series=pd.Series(x_prediction_maximum_power_ann)
    y_real_maximum_power_series=pd.Series(y_real_maximum_power)
    corr_ann = round(x_prediction_maximum_power_ann_series.corr(y_real_maximum_power_series), 4)
    rmse_val= rmse(x_prediction_maximum_power_ann,y_real_maximum_power)
    print('ANN,R2',corr_ann,'RMSE',rmse_val)
    layer_str = '%d'%2
    neurons_str = '%d'%neurons
    corr_ann_str = '%f'%corr_ann
    rmse_val_str = '%f'%rmse_val
    file.write(layer_str+' '+ neurons_str +' '+ corr_ann_str+' '+ rmse_val_str +'\n')
file.close()

file = open('../../results/ANN_Opt_layer/alkaline-ANN-3-layer.txt','w')
dropout_list= np.arange(0, 1, 0.1)
for neurons in range(50,1050,50):
    ###########implementing hyperparameters##########
    neurons1=neurons
    ###########keras ANN model construction##########
    model = Sequential() 
    model.add(Dense(neurons1, input_dim=121, kernel_initializer='random_normal',
                bias_initializer='random_normal',activation='relu',kernel_regularizer=keras.regularizers.l2(0.0001))) 
    model.add(Dropout(0))
    model.add(Dense(neurons1, input_dim=neurons1, kernel_initializer='random_normal',
                bias_initializer='random_normal',activation='relu',kernel_regularizer=keras.regularizers.l2(0.0001))) 
    model.add(Dropout(0))
    model.add(Dense(neurons1, input_dim=neurons1, kernel_initializer='random_normal',
                bias_initializer='random_normal',activation='relu',kernel_regularizer=keras.regularizers.l2(0.0001))) 
    model.add(Dropout(0))
    model.add(Dense(neurons1, input_dim=neurons1, kernel_initializer='random_normal',
                bias_initializer='random_normal',activation='relu',kernel_regularizer=keras.regularizers.l2(0.0001))) 
    model.add(Dropout(0))
    model.add(Dense(1, input_dim=neurons1, activation='linear'))
    adam=tf.keras.optimizers.Adam(lr=0.002)
    model.compile(loss='mse', optimizer=adam) 
    #print('Training ------------')
    ###########train the model with the training set##########
    ###########testset has been remove before##########
    model.fit(X_train, y_train,verbose=0, epochs=200, batch_size=32,validation_split=0.1,callbacks=[TensorBoard(log_dir='mytensorboard')])
    loss= model.evaluate(X_test, y_test)
    predict_ann= model.predict(X_test)
    train_ann= model.predict(X_train)
    ###########result output##########
    x_prediction_maximum_power_ann=predict_ann*np.std(data,axis=0)[121]+np.mean(data,axis=0)[121]
    y_real_maximum_power=y_test*np.std(data,axis=0)[121]+np.mean(data,axis=0)[121]
    x_prediction_maximum_power_ann=x_prediction_maximum_power_ann[:,0]
    x_prediction_maximum_power_ann_series=pd.Series(x_prediction_maximum_power_ann)
    y_real_maximum_power_series=pd.Series(y_real_maximum_power)
    corr_ann = round(x_prediction_maximum_power_ann_series.corr(y_real_maximum_power_series), 4)
    rmse_val= rmse(x_prediction_maximum_power_ann,y_real_maximum_power)
    print(neurons,'ANN,R2',corr_ann,'RMSE',rmse_val)
    layer_str = '%d'%3
    neurons_str = '%d'%neurons
    corr_ann_str = '%f'%corr_ann
    rmse_val_str = '%f'%rmse_val
    file.write(layer_str+' '+ neurons_str +' '+ corr_ann_str+' '+ rmse_val_str +'\n')
file.close()

file = open('../../results/ANN_Opt_layer/alkaline-ANN-4-layer.txt','w')
dropout_list= np.arange(0, 1, 0.1)
for neurons in range(50,1050,50):
    ###########implementing hyperparameters##########
    neurons1=neurons
    ###########keras ANN model construction##########
    model = Sequential() 
    model.add(Dense(neurons1, input_dim=121, kernel_initializer='random_normal',
                bias_initializer='random_normal',activation='relu',kernel_regularizer=keras.regularizers.l2(0.0001))) 
    model.add(Dropout(0))
    model.add(Dense(neurons1, input_dim=neurons1, kernel_initializer='random_normal',
                bias_initializer='random_normal',activation='relu',kernel_regularizer=keras.regularizers.l2(0.0001))) 
    model.add(Dropout(0))
    model.add(Dense(neurons1, input_dim=neurons1, kernel_initializer='random_normal',
                bias_initializer='random_normal',activation='relu',kernel_regularizer=keras.regularizers.l2(0.0001))) 
    model.add(Dropout(0))
    model.add(Dense(neurons1, input_dim=neurons1, kernel_initializer='random_normal',
                bias_initializer='random_normal',activation='relu',kernel_regularizer=keras.regularizers.l2(0.0001))) 
    model.add(Dropout(0))
    model.add(Dense(neurons1, input_dim=neurons1, kernel_initializer='random_normal',
                bias_initializer='random_normal',activation='relu',kernel_regularizer=keras.regularizers.l2(0.0001))) 
    model.add(Dropout(0))
    model.add(Dense(1, input_dim=neurons1, activation='linear'))
    adam=tf.keras.optimizers.Adam(lr=0.002)
    model.compile(loss='mse', optimizer=adam) 
    #print('Training ------------')
    ###########train the model with the training set##########
    ###########testset has been remove before##########
    model.fit(X_train, y_train,verbose=0, epochs=200, batch_size=32,validation_split=0.1,callbacks=[TensorBoard(log_dir='mytensorboard')])
    loss= model.evaluate(X_test, y_test)
    predict_ann= model.predict(X_test)
    train_ann= model.predict(X_train)
    ###########result output##########
    x_prediction_maximum_power_ann=predict_ann*np.std(data,axis=0)[121]+np.mean(data,axis=0)[121]
    y_real_maximum_power=y_test*np.std(data,axis=0)[121]+np.mean(data,axis=0)[121]
    x_prediction_maximum_power_ann=x_prediction_maximum_power_ann[:,0]
    x_prediction_maximum_power_ann_series=pd.Series(x_prediction_maximum_power_ann)
    y_real_maximum_power_series=pd.Series(y_real_maximum_power)
    corr_ann = round(x_prediction_maximum_power_ann_series.corr(y_real_maximum_power_series), 4)
    rmse_val= rmse(x_prediction_maximum_power_ann,y_real_maximum_power)
    print(neurons,'ANN,R2',corr_ann,'RMSE',rmse_val)
    layer_str = '%d'%4
    neurons_str = '%d'%neurons
    corr_ann_str = '%f'%corr_ann
    rmse_val_str = '%f'%rmse_val
    file.write(layer_str+' '+ neurons_str +' '+ corr_ann_str+' '+ rmse_val_str +'\n')
file.close()

file = open('../../results/ANN_Opt_layer/alkaline-ANN-5-layer.txt','w')
dropout_list= np.arange(0, 1, 0.1)
for neurons in range(50,1050,50):
    ###########implementing hyperparameters##########
    neurons1=neurons
    ###########keras ANN model construction##########
    model = Sequential() 
    model.add(Dense(neurons1, input_dim=121, kernel_initializer='random_normal',
                bias_initializer='random_normal',activation='relu',kernel_regularizer=keras.regularizers.l2(0.0001))) 
    model.add(Dropout(0))
    model.add(Dense(neurons1, input_dim=neurons1, kernel_initializer='random_normal',
                bias_initializer='random_normal',activation='relu',kernel_regularizer=keras.regularizers.l2(0.0001))) 
    model.add(Dropout(0))
    model.add(Dense(neurons1, input_dim=neurons1, kernel_initializer='random_normal',
                bias_initializer='random_normal',activation='relu',kernel_regularizer=keras.regularizers.l2(0.0001))) 
    model.add(Dropout(0))
    model.add(Dense(neurons1, input_dim=neurons1, kernel_initializer='random_normal',
                bias_initializer='random_normal',activation='relu',kernel_regularizer=keras.regularizers.l2(0.0001))) 
    model.add(Dropout(0))
    model.add(Dense(neurons1, input_dim=neurons1, kernel_initializer='random_normal',
                bias_initializer='random_normal',activation='relu',kernel_regularizer=keras.regularizers.l2(0.0001))) 
    model.add(Dropout(0))
    model.add(Dense(neurons1, input_dim=neurons1, kernel_initializer='random_normal',
                bias_initializer='random_normal',activation='relu',kernel_regularizer=keras.regularizers.l2(0.0001))) 
    model.add(Dropout(0))
    model.add(Dense(1, input_dim=neurons1, activation='linear'))
    adam=tf.keras.optimizers.Adam(lr=0.002)
    model.compile(loss='mse', optimizer=adam) 
    #print('Training ------------')
    ###########train the model with the training set##########
    ###########testset has been remove before##########
    model.fit(X_train, y_train,verbose=0, epochs=200, batch_size=32,validation_split=0.1,callbacks=[TensorBoard(log_dir='mytensorboard')])
    loss= model.evaluate(X_test, y_test)
    predict_ann= model.predict(X_test)
    train_ann= model.predict(X_train)
    ###########result output##########
    x_prediction_maximum_power_ann=predict_ann*np.std(data,axis=0)[121]+np.mean(data,axis=0)[121]
    y_real_maximum_power=y_test*np.std(data,axis=0)[121]+np.mean(data,axis=0)[121]
    x_prediction_maximum_power_ann=x_prediction_maximum_power_ann[:,0]
    x_prediction_maximum_power_ann_series=pd.Series(x_prediction_maximum_power_ann)
    y_real_maximum_power_series=pd.Series(y_real_maximum_power)
    corr_ann = round(x_prediction_maximum_power_ann_series.corr(y_real_maximum_power_series), 4)
    rmse_val= rmse(x_prediction_maximum_power_ann,y_real_maximum_power)
    print(neurons,'ANN,R2',corr_ann,'RMSE',rmse_val)
    layer_str = '%d'%5
    neurons_str = '%d'%neurons
    corr_ann_str = '%f'%corr_ann
    rmse_val_str = '%f'%rmse_val
    file.write(layer_str+' '+ neurons_str +' '+ corr_ann_str+' '+ rmse_val_str +'\n')
file.close()

file = open('../../results/ANN_Opt_layer/alkaline-ANN-6-layer.txt','w')
dropout_list= np.arange(0, 1, 0.1)
for neurons in range(50,1050,50):
    ###########implementing hyperparameters##########
    neurons1=neurons
    ###########keras ANN model construction##########
    model = Sequential() 
    model.add(Dense(neurons1, input_dim=121, kernel_initializer='random_normal',
                bias_initializer='random_normal',activation='relu',kernel_regularizer=keras.regularizers.l2(0.0001))) 
    model.add(Dropout(0))
    model.add(Dense(neurons1, input_dim=neurons1, kernel_initializer='random_normal',
                bias_initializer='random_normal',activation='relu',kernel_regularizer=keras.regularizers.l2(0.0001))) 
    model.add(Dropout(0))
    model.add(Dense(neurons1, input_dim=neurons1, kernel_initializer='random_normal',
                bias_initializer='random_normal',activation='relu',kernel_regularizer=keras.regularizers.l2(0.0001))) 
    model.add(Dropout(0))
    model.add(Dense(neurons1, input_dim=neurons1, kernel_initializer='random_normal',
                bias_initializer='random_normal',activation='relu',kernel_regularizer=keras.regularizers.l2(0.0001))) 
    model.add(Dropout(0))
    model.add(Dense(neurons1, input_dim=neurons1, kernel_initializer='random_normal',
                bias_initializer='random_normal',activation='relu',kernel_regularizer=keras.regularizers.l2(0.0001))) 
    model.add(Dropout(0))
    model.add(Dense(neurons1, input_dim=neurons1, kernel_initializer='random_normal',
                bias_initializer='random_normal',activation='relu',kernel_regularizer=keras.regularizers.l2(0.0001))) 
    model.add(Dropout(0))
    model.add(Dense(neurons1, input_dim=neurons1, kernel_initializer='random_normal',
                bias_initializer='random_normal',activation='relu',kernel_regularizer=keras.regularizers.l2(0.0001))) 
    model.add(Dropout(0))
    model.add(Dense(1, input_dim=neurons1, activation='linear'))
    adam=tf.keras.optimizers.Adam(lr=0.002)
    model.compile(loss='mse', optimizer=adam) 
    #print('Training ------------')
    ###########train the model with the training set##########
    ###########testset has been remove before##########
    model.fit(X_train, y_train,verbose=0, epochs=200, batch_size=32,validation_split=0.1,callbacks=[TensorBoard(log_dir='mytensorboard')])
    loss= model.evaluate(X_test, y_test)
    predict_ann= model.predict(X_test)
    train_ann= model.predict(X_train)
    ###########result output##########
    x_prediction_maximum_power_ann=predict_ann*np.std(data,axis=0)[121]+np.mean(data,axis=0)[121]
    y_real_maximum_power=y_test*np.std(data,axis=0)[121]+np.mean(data,axis=0)[121]
    x_prediction_maximum_power_ann=x_prediction_maximum_power_ann[:,0]
    x_prediction_maximum_power_ann_series=pd.Series(x_prediction_maximum_power_ann)
    y_real_maximum_power_series=pd.Series(y_real_maximum_power)
    corr_ann = round(x_prediction_maximum_power_ann_series.corr(y_real_maximum_power_series), 4)
    rmse_val= rmse(x_prediction_maximum_power_ann,y_real_maximum_power)
    print(neurons,'ANN,R2',corr_ann,'RMSE',rmse_val)
    layer_str = '%d'%6
    neurons_str = '%d'%neurons
    corr_ann_str = '%f'%corr_ann
    rmse_val_str = '%f'%rmse_val
    file.write(layer_str+' '+ neurons_str +' '+ corr_ann_str+' '+ rmse_val_str +'\n')
file.close()

file = open('../../results/ANN_Opt_layer/alkaline-ANN-7-layer.txt','w')
dropout_list= np.arange(0, 1, 0.1)
for neurons in range(50,1050,50):
    ###########implementing hyperparameters##########
    neurons1=neurons
    ###########keras ANN model construction##########
    model = Sequential() 
    model.add(Dense(neurons1, input_dim=121, kernel_initializer='random_normal',
                bias_initializer='random_normal',activation='relu',kernel_regularizer=keras.regularizers.l2(0.0001))) 
    model.add(Dropout(0))
    model.add(Dense(neurons1, input_dim=neurons1, kernel_initializer='random_normal',
                bias_initializer='random_normal',activation='relu',kernel_regularizer=keras.regularizers.l2(0.0001))) 
    model.add(Dropout(0))
    model.add(Dense(neurons1, input_dim=neurons1, kernel_initializer='random_normal',
                bias_initializer='random_normal',activation='relu',kernel_regularizer=keras.regularizers.l2(0.0001))) 
    model.add(Dropout(0))
    model.add(Dense(neurons1, input_dim=neurons1, kernel_initializer='random_normal',
                bias_initializer='random_normal',activation='relu',kernel_regularizer=keras.regularizers.l2(0.0001))) 
    model.add(Dropout(0))
    model.add(Dense(neurons1, input_dim=neurons1, kernel_initializer='random_normal',
                bias_initializer='random_normal',activation='relu',kernel_regularizer=keras.regularizers.l2(0.0001))) 
    model.add(Dropout(0))
    model.add(Dense(neurons1, input_dim=neurons1, kernel_initializer='random_normal',
                bias_initializer='random_normal',activation='relu',kernel_regularizer=keras.regularizers.l2(0.0001))) 
    model.add(Dropout(0))
    model.add(Dense(neurons1, input_dim=neurons1, kernel_initializer='random_normal',
                bias_initializer='random_normal',activation='relu',kernel_regularizer=keras.regularizers.l2(0.0001))) 
    model.add(Dropout(0))
    model.add(Dense(neurons1, input_dim=neurons1, kernel_initializer='random_normal',
                bias_initializer='random_normal',activation='relu',kernel_regularizer=keras.regularizers.l2(0.0001))) 
    model.add(Dropout(0))
    model.add(Dense(1, input_dim=neurons1, activation='linear'))
    adam=tf.keras.optimizers.Adam(lr=0.002)
    model.compile(loss='mse', optimizer=adam) 
    #print('Training ------------')
    ###########train the model with the training set##########
    ###########testset has been remove before##########
    model.fit(X_train, y_train,verbose=0, epochs=200, batch_size=32,validation_split=0.1,callbacks=[TensorBoard(log_dir='mytensorboard')])
    loss= model.evaluate(X_test, y_test)
    predict_ann= model.predict(X_test)
    train_ann= model.predict(X_train)
    ###########result output##########
    x_prediction_maximum_power_ann=predict_ann*np.std(data,axis=0)[121]+np.mean(data,axis=0)[121]
    y_real_maximum_power=y_test*np.std(data,axis=0)[121]+np.mean(data,axis=0)[121]
    x_prediction_maximum_power_ann=x_prediction_maximum_power_ann[:,0]
    x_prediction_maximum_power_ann_series=pd.Series(x_prediction_maximum_power_ann)
    y_real_maximum_power_series=pd.Series(y_real_maximum_power)
    corr_ann = round(x_prediction_maximum_power_ann_series.corr(y_real_maximum_power_series), 4)
    rmse_val= rmse(x_prediction_maximum_power_ann,y_real_maximum_power)
    print(neurons,'ANN,R2',corr_ann,'RMSE',rmse_val)
    layer_str = '%d'%7
    neurons_str = '%d'%neurons
    corr_ann_str = '%f'%corr_ann
    rmse_val_str = '%f'%rmse_val
    file.write(layer_str+' '+ neurons_str +' '+ corr_ann_str+' '+ rmse_val_str +'\n')
file.close()

file = open('../../results/ANN_Opt_layer/alkaline-ANN-8-layer.txt','w')
dropout_list= np.arange(0, 1, 0.1)
for neurons in range(50,1050,50):
    ###########implementing hyperparameters##########
    neurons1=neurons
    ###########keras ANN model construction##########
    model = Sequential() 
    model.add(Dense(neurons1, input_dim=121, kernel_initializer='random_normal',
                bias_initializer='random_normal',activation='relu',kernel_regularizer=keras.regularizers.l2(0.0001))) 
    model.add(Dropout(0))
    model.add(Dense(neurons1, input_dim=neurons1, kernel_initializer='random_normal',
                bias_initializer='random_normal',activation='relu',kernel_regularizer=keras.regularizers.l2(0.0001))) 
    model.add(Dropout(0))
    model.add(Dense(neurons1, input_dim=neurons1, kernel_initializer='random_normal',
                bias_initializer='random_normal',activation='relu',kernel_regularizer=keras.regularizers.l2(0.0001))) 
    model.add(Dropout(0))
    model.add(Dense(neurons1, input_dim=neurons1, kernel_initializer='random_normal',
                bias_initializer='random_normal',activation='relu',kernel_regularizer=keras.regularizers.l2(0.0001))) 
    model.add(Dropout(0))
    model.add(Dense(neurons1, input_dim=neurons1, kernel_initializer='random_normal',
                bias_initializer='random_normal',activation='relu',kernel_regularizer=keras.regularizers.l2(0.0001))) 
    model.add(Dropout(0))
    model.add(Dense(neurons1, input_dim=neurons1, kernel_initializer='random_normal',
                bias_initializer='random_normal',activation='relu',kernel_regularizer=keras.regularizers.l2(0.0001))) 
    model.add(Dropout(0))
    model.add(Dense(neurons1, input_dim=neurons1, kernel_initializer='random_normal',
                bias_initializer='random_normal',activation='relu',kernel_regularizer=keras.regularizers.l2(0.0001))) 
    model.add(Dropout(0))
    model.add(Dense(neurons1, input_dim=neurons1, kernel_initializer='random_normal',
                bias_initializer='random_normal',activation='relu',kernel_regularizer=keras.regularizers.l2(0.0001))) 
    model.add(Dropout(0))
    model.add(Dense(neurons1, input_dim=neurons1, kernel_initializer='random_normal',
                bias_initializer='random_normal',activation='relu',kernel_regularizer=keras.regularizers.l2(0.0001))) 
    model.add(Dropout(0))
    model.add(Dense(1, input_dim=neurons1, activation='linear'))
    adam=tf.keras.optimizers.Adam(lr=0.002)
    model.compile(loss='mse', optimizer=adam) 
    #print('Training ------------')
    ###########train the model with the training set##########
    ###########testset has been remove before##########
    model.fit(X_train, y_train,verbose=0, epochs=200, batch_size=32,validation_split=0.1,callbacks=[TensorBoard(log_dir='mytensorboard')])
    loss= model.evaluate(X_test, y_test)
    predict_ann= model.predict(X_test)
    train_ann= model.predict(X_train)
    ###########result output##########
    x_prediction_maximum_power_ann=predict_ann*np.std(data,axis=0)[121]+np.mean(data,axis=0)[121]
    y_real_maximum_power=y_test*np.std(data,axis=0)[121]+np.mean(data,axis=0)[121]
    x_prediction_maximum_power_ann=x_prediction_maximum_power_ann[:,0]
    x_prediction_maximum_power_ann_series=pd.Series(x_prediction_maximum_power_ann)
    y_real_maximum_power_series=pd.Series(y_real_maximum_power)
    corr_ann = round(x_prediction_maximum_power_ann_series.corr(y_real_maximum_power_series), 4)
    rmse_val= rmse(x_prediction_maximum_power_ann,y_real_maximum_power)
    print(neurons,'ANN,R2',corr_ann,'RMSE',rmse_val)
    layer_str = '%d'%8
    neurons_str = '%d'%neurons
    corr_ann_str = '%f'%corr_ann
    rmse_val_str = '%f'%rmse_val
    file.write(layer_str+' '+ neurons_str +' '+ corr_ann_str+' '+ rmse_val_str +'\n')
file.close()

file = open('../../results/ANN_Opt_layer/alkaline-ANN-9-layer.txt','w')
dropout_list= np.arange(0, 1, 0.1)
for neurons in range(50,1050,50):
    ###########implementing hyperparameters##########
    neurons1=neurons
    ###########keras ANN model construction##########
    model = Sequential() 
    model.add(Dense(neurons1, input_dim=121, kernel_initializer='random_normal',
                bias_initializer='random_normal',activation='relu',kernel_regularizer=keras.regularizers.l2(0.0001))) 
    model.add(Dropout(0))
    model.add(Dense(neurons1, input_dim=neurons1, kernel_initializer='random_normal',
                bias_initializer='random_normal',activation='relu',kernel_regularizer=keras.regularizers.l2(0.0001))) 
    model.add(Dropout(0))
    model.add(Dense(neurons1, input_dim=neurons1, kernel_initializer='random_normal',
                bias_initializer='random_normal',activation='relu',kernel_regularizer=keras.regularizers.l2(0.0001))) 
    model.add(Dropout(0))
    model.add(Dense(neurons1, input_dim=neurons1, kernel_initializer='random_normal',
                bias_initializer='random_normal',activation='relu',kernel_regularizer=keras.regularizers.l2(0.0001))) 
    model.add(Dropout(0))
    model.add(Dense(neurons1, input_dim=neurons1, kernel_initializer='random_normal',
                bias_initializer='random_normal',activation='relu',kernel_regularizer=keras.regularizers.l2(0.0001))) 
    model.add(Dropout(0))
    model.add(Dense(neurons1, input_dim=neurons1, kernel_initializer='random_normal',
                bias_initializer='random_normal',activation='relu',kernel_regularizer=keras.regularizers.l2(0.0001))) 
    model.add(Dropout(0))
    model.add(Dense(neurons1, input_dim=neurons1, kernel_initializer='random_normal',
                bias_initializer='random_normal',activation='relu',kernel_regularizer=keras.regularizers.l2(0.0001))) 
    model.add(Dropout(0))
    model.add(Dense(neurons1, input_dim=neurons1, kernel_initializer='random_normal',
                bias_initializer='random_normal',activation='relu',kernel_regularizer=keras.regularizers.l2(0.0001))) 
    model.add(Dropout(0))
    model.add(Dense(neurons1, input_dim=neurons1, kernel_initializer='random_normal',
                bias_initializer='random_normal',activation='relu',kernel_regularizer=keras.regularizers.l2(0.0001))) 
    model.add(Dropout(0))
    model.add(Dense(neurons1, input_dim=neurons1, kernel_initializer='random_normal',
                bias_initializer='random_normal',activation='relu',kernel_regularizer=keras.regularizers.l2(0.0001))) 
    model.add(Dropout(0))
    model.add(Dense(1, input_dim=neurons1, activation='linear'))
    adam=tf.keras.optimizers.Adam(lr=0.002)
    model.compile(loss='mse', optimizer=adam) 
    #print('Training ------------')
    ###########train the model with the training set##########
    ###########testset has been remove before##########
    model.fit(X_train, y_train,verbose=0, epochs=200, batch_size=32,validation_split=0.1,callbacks=[TensorBoard(log_dir='mytensorboard')])
    loss= model.evaluate(X_test, y_test)
    predict_ann= model.predict(X_test)
    train_ann= model.predict(X_train)
    ###########result output##########
    x_prediction_maximum_power_ann=predict_ann*np.std(data,axis=0)[121]+np.mean(data,axis=0)[121]
    y_real_maximum_power=y_test*np.std(data,axis=0)[121]+np.mean(data,axis=0)[121]
    x_prediction_maximum_power_ann=x_prediction_maximum_power_ann[:,0]
    x_prediction_maximum_power_ann_series=pd.Series(x_prediction_maximum_power_ann)
    y_real_maximum_power_series=pd.Series(y_real_maximum_power)
    corr_ann = round(x_prediction_maximum_power_ann_series.corr(y_real_maximum_power_series), 4)
    rmse_val= rmse(x_prediction_maximum_power_ann,y_real_maximum_power)
    print(neurons,'ANN,R2',corr_ann,'RMSE',rmse_val)
    layer_str = '%d'%9
    neurons_str = '%d'%neurons
    corr_ann_str = '%f'%corr_ann
    rmse_val_str = '%f'%rmse_val
    file.write(layer_str+' '+ neurons_str +' '+ corr_ann_str+' '+ rmse_val_str +'\n')
file.close()

file = open('../../results/ANN_Opt_layer/alkaline-ANN-10-layer.txt','w')
dropout_list= np.arange(0, 1, 0.1)
for neurons in range(50,1050,50):
    ###########implementing hyperparameters##########
    neurons1=neurons
    ###########keras ANN model construction##########
    model = Sequential() 
    model.add(Dense(neurons1, input_dim=121, kernel_initializer='random_normal',
                bias_initializer='random_normal',activation='relu',kernel_regularizer=keras.regularizers.l2(0.0001))) 
    model.add(Dropout(0))
    model.add(Dense(neurons1, input_dim=neurons1, kernel_initializer='random_normal',
                bias_initializer='random_normal',activation='relu',kernel_regularizer=keras.regularizers.l2(0.0001))) 
    model.add(Dropout(0))
    model.add(Dense(neurons1, input_dim=neurons1, kernel_initializer='random_normal',
                bias_initializer='random_normal',activation='relu',kernel_regularizer=keras.regularizers.l2(0.0001))) 
    model.add(Dropout(0))
    model.add(Dense(neurons1, input_dim=neurons1, kernel_initializer='random_normal',
                bias_initializer='random_normal',activation='relu',kernel_regularizer=keras.regularizers.l2(0.0001))) 
    model.add(Dropout(0))
    model.add(Dense(neurons1, input_dim=neurons1, kernel_initializer='random_normal',
                bias_initializer='random_normal',activation='relu',kernel_regularizer=keras.regularizers.l2(0.0001))) 
    model.add(Dropout(0))
    model.add(Dense(neurons1, input_dim=neurons1, kernel_initializer='random_normal',
                bias_initializer='random_normal',activation='relu',kernel_regularizer=keras.regularizers.l2(0.0001))) 
    model.add(Dropout(0))
    model.add(Dense(neurons1, input_dim=neurons1, kernel_initializer='random_normal',
                bias_initializer='random_normal',activation='relu',kernel_regularizer=keras.regularizers.l2(0.0001))) 
    model.add(Dropout(0))
    model.add(Dense(neurons1, input_dim=neurons1, kernel_initializer='random_normal',
                bias_initializer='random_normal',activation='relu',kernel_regularizer=keras.regularizers.l2(0.0001))) 
    model.add(Dropout(0))
    model.add(Dense(neurons1, input_dim=neurons1, kernel_initializer='random_normal',
                bias_initializer='random_normal',activation='relu',kernel_regularizer=keras.regularizers.l2(0.0001))) 
    model.add(Dropout(0))
    model.add(Dense(neurons1, input_dim=neurons1, kernel_initializer='random_normal',
                bias_initializer='random_normal',activation='relu',kernel_regularizer=keras.regularizers.l2(0.0001))) 
    model.add(Dropout(0))
    model.add(Dense(neurons1, input_dim=neurons1, kernel_initializer='random_normal',
                bias_initializer='random_normal',activation='relu',kernel_regularizer=keras.regularizers.l2(0.0001))) 
    model.add(Dropout(0))
    model.add(Dense(1, input_dim=neurons1, activation='linear'))
    adam=tf.keras.optimizers.Adam(lr=0.002)
    model.compile(loss='mse', optimizer=adam) 
    #print('Training ------------')
    ###########train the model with the training set##########
    ###########testset has been remove before##########
    model.fit(X_train, y_train,verbose=0, epochs=200, batch_size=32,validation_split=0.1,callbacks=[TensorBoard(log_dir='mytensorboard')])
    loss= model.evaluate(X_test, y_test)
    predict_ann= model.predict(X_test)
    train_ann= model.predict(X_train)
    ###########result output##########
    x_prediction_maximum_power_ann=predict_ann*np.std(data,axis=0)[121]+np.mean(data,axis=0)[121]
    y_real_maximum_power=y_test*np.std(data,axis=0)[121]+np.mean(data,axis=0)[121]
    x_prediction_maximum_power_ann=x_prediction_maximum_power_ann[:,0]
    x_prediction_maximum_power_ann_series=pd.Series(x_prediction_maximum_power_ann)
    y_real_maximum_power_series=pd.Series(y_real_maximum_power)
    corr_ann = round(x_prediction_maximum_power_ann_series.corr(y_real_maximum_power_series), 4)
    rmse_val= rmse(x_prediction_maximum_power_ann,y_real_maximum_power)
    print(neurons,'ANN,R2',corr_ann,'RMSE',rmse_val)
    layer_str = '%d'%10
    neurons_str = '%d'%neurons
    corr_ann_str = '%f'%corr_ann
    rmse_val_str = '%f'%rmse_val
    file.write(layer_str+' '+ neurons_str +' '+ corr_ann_str+' '+ rmse_val_str +'\n')
file.close()